In [1]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

In [9]:
from Dataset import load_dataset_from_pickle, MILDataset
import utils
from backbone import BackboneResNet

In [3]:
pkl_path='../re_pickle_train_patch'
EPOCH=10
device = 'cuda' if torch.cuda.is_available() else 'cpu'
TRAIN_CSV_PATH = '../total_split_train_val/fold_4_train.csv'
VALID_CSV_PATH = '../total_split_train_val/fold_4_val.csv'

In [4]:
slide_name = utils.get_slides_name(TRAIN_CSV_PATH)

In [5]:
pkl_path = utils.get_image_path(image_path='../re_pickle_train_patch',slide_name_list=slide_name)

In [6]:
BATCH_SIZE=700

In [10]:
feature_extractor = BackboneResNet('resnet18')

Feature extractor: resnet18


In [50]:
num_cluster = 11

In [84]:
def visualize_images(features, patches, labels, num_patches, num_cluster, image_size=(1.5, 1.5)) :
    num_cluster_per_patches = {}
    rows = 1
    cols = min(num_patches, 6)
    for cluster_index in range(num_cluster) :
        cluster_indices = np.where(labels==cluster_index)[0]
        # print(cluster_indices)
        random_indices = np.random.choice(cluster_indices, num_patches, replace=False)
        num_cluster_patches = len(cluster_indices)
        num_cluster_per_patches[cluster_index] = num_cluster_patches
        
        fig, axes = plt.subplots(rows, cols, figsize=(image_size[1] * cols, image_size[0] * rows))
        for i, index in enumerate(random_indices) :
            ax = axes[i]
            ax.imshow(patches[index].transpose(1, 2, 0))
            ax.set_title(f'label {cluster_index}, Image {index}')
            ax.axis('off')
        plt.tight_layout()
        plt.show()
    print(num_cluster_per_patches)
          

In [ ]:
for train_wsi_path in pkl_path:
    print(train_wsi_path)
    train_wsi = load_dataset_from_pickle(train_wsi_path)
    train_wsi_data_loader = DataLoader(train_wsi, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
    with torch.no_grad() :
        for batch_index, (wsi_patch, wsi_label) in enumerate(train_wsi_data_loader) :
            batch_size = wsi_patch.size(0)
            if (batch_index > 0) & (batch_size < 200) :
                break    
            features = feature_extractor(wsi_patch)
            wsi_patch_np = wsi_patch.numpy()
            cluster = KMeans(n_clusters=num_cluster, n_init=15)
            cluster.fit(features)

            # print(cluster)
            result_cluster = features.detach().numpy().copy()
            # result_cluster['cluster'] = 
            visualize_images(features=result_cluster, patches=wsi_patch_np, labels=cluster.labels_, num_patches=6, num_cluster=num_cluster)
        
    break

In [87]:
!nvidia-smi

Mon Jan 22 12:05:27 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:48:00.0 Off |                  Off |
| 30%   36C    P8    29W / 300W |      3MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------